# 무역 데이터 EDA (HS4 / item_id 단위 EDA 및 micro-level deep dive)

In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns


df_train = pd.read_csv('train.csv')
print(df_train.head())
df_submission = pd.read_csv('sample_submission.csv')
print(df_submission.head())



    item_id  year  month  seq  type   hs4    weight  quantity     value
0  DEWLVASR  2022      1  1.0     1  3038   14858.0       0.0   32688.0
1  ELQGMQWE  2022      1  1.0     1  2002   62195.0       0.0  110617.0
2  AHMDUILJ  2022      1  1.0     1  2102   18426.0       0.0   72766.0
3  XIPPENFQ  2022      1  1.0     1  2501   20426.0       0.0   11172.0
4  FTSVTTSR  2022      1  1.0     1  2529  248000.0       0.0  143004.0
  leading_item_id following_item_id      value
0        DEWLVASR          ELQGMQWE  999999999
1        DEWLVASR          AHMDUILJ  999999999
2        DEWLVASR          XIPPENFQ  999999999
3        DEWLVASR          FTSVTTSR  999999999
4        DEWLVASR          XMKRPGLB  999999999


## 1. 전처리

- zero / missing flag 생성:
  - value, weight, quantity의 0은 정상적일 수 있음 👉 제거 X
  - 대신 zero 패턴 자체를 feature로 인식하기 위해 flag 생성.

- 결측치 처리:
  - weight, quantity, value 결측은 보간하지 않음 (의미 있는 결측이기 때문)

- item_id / HS4 단위 monthly pivot 생성
  - → Comovement 분석의 기본 단위로 사용될 시계열 구조 확보.

In [14]:
# -----------------------------
# 2. Create date column
# Why? 시계열 EDA 및 lag-lead 분석에 필수
# -----------------------------
df_train["date"] = df_train["year"].astype(str) + "-" + df_train["month"].astype(str)
df_train["date"] = pd.to_datetime(df_train["date"]).dt.to_period("M")

# -----------------------------
# 3. Ensure numeric type
# -----------------------------
for col in ['value','weight','quantity']:
    df_train[col] = pd.to_numeric(df_train[col], errors='coerce')

# -----------------------------
# 4. Zero-aware preprocessing
# Zero는 제거하지 않고 flag로 보존 (패턴 분석용)
# -----------------------------
df_train['value_zero'] = (df_train['value'] == 0).astype(int)
df_train['weight_zero'] = (df_train['weight'] == 0).astype(int)
df_train['quantity_zero'] = (df_train['quantity'] == 0).astype(int)


df_train.head(20)

,item_id,year,month,seq,type,hs4,weight,quantity,value,value_zero,weight_zero,quantity_zero,value_missing,weight_missing,quantity_missing,date
0,DEWLVASR,2022,1,1.0,1,3038,14858.0,0.0,32688.0,0,0,1,0,0,0,2022-01
1,ELQGMQWE,2022,1,1.0,1,2002,62195.0,0.0,110617.0,0,0,1,0,0,0,2022-01
2,AHMDUILJ,2022,1,1.0,1,2102,18426.0,0.0,72766.0,0,0,1,0,0,0,2022-01
3,XIPPENFQ,2022,1,1.0,1,2501,20426.0,0.0,11172.0,0,0,1,0,0,0,2022-01
4,FTSVTTSR,2022,1,1.0,1,2529,248000.0,0.0,143004.0,0,0,1,0,0,0,2022-01
5,XMKRPGLB,2022,1,1.0,1,2529,558000.0,0.0,205676.0,0,0,1,0,0,0,2022-01
6,RCBZUSIM,2022,1,1.0,1,2805,0.0,0.0,459.0,0,1,1,0,0,0,2022-01
7,SUOYXCHP,2022,1,1.0,1,2805,0.0,0.0,482.0,0,1,1,0,0,0,2022-01
8,ZKENOUDA,2022,1,1.0,1,2805,2.0,0.0,2783.0,0,0,1,0,0,0,2022-01
9,WQMVCOEM,2022,1,1.0,1,2805,187.0,0.0,217777.0,0,0,1,0,0,0,2022-01


In [10]:
df_train['month'].value_counts()

month
7     1033
2     1022
4     1008
5     1004
6     1000
3      997
1      991
12     777
11     761
9      757
8      743
10     743
Name: count, dtype: int64

### 모든 item_id × 모든 month 패널 완성(Panel Completion)

현재 데이터는 모든 item_id가 모든 월에 등장하지 않음 → 공행성 분석에서 큰 문제됨.
동시에 존재하는 시계열끼리 비교해야 함

예:
- A item은 2022~2025 전 기간 존재
- B item은 특정 계절만 존재
- C item은 중간에 구멍 있음

이런 상태에서 comovement correlation 계산하면 시계열 길이가 불일치 → 오류 / 왜곡 발생, 상관계수 계산 불가<br>
- A : 2022.01 ~ 2025.07 모든 달에 존재 = 42개월
- C : 2022.03 ~ 2023.05 일부 달에 존재 = 15개월
→ 반드시 해야 하는 전처리:<br>
모든 item_id × 모든 month 조합을 만들고 없는 row는 0 또는 NaN으로 채우기

In [15]:
# 모든 월 범위 생성
full_months = pd.date_range(df_train['date'].min(), df_train['date'].max(), freq="MS")

# 모든 item_id × 모든 month
panel = pd.MultiIndex.from_product(
    [df_train['item_id'].unique(), full_months],
    names=['item_id','date']
).to_frame(index=False)

# merge to complete panel
df_panel = panel.merge(df_train, on=['item_id','date'], how='left')

df_panel.head()

ValueError: Cannot convert Period to Timestamp unambiguously. Use to_timestamp

## 2. HS4 / item_id 군집성 분석

In [ ]:
hs4_summary = df.groupby('hs4')[['value','weight']].mean().reset_index()
hs4_qty_rate = (
    df.assign(qty_valid = df['quantity'].notna() & (df['quantity']>0))
      .groupby('hs4')['qty_valid']
      .mean()
      .reset_index(name='quantity_usage_rate')
)
hs4_summary.head(), hs4_qty_rate.head()

## 3. item_id Deep Dive

In [ ]:
sample_items = np.random.choice(df['item_id'].unique(), size=8, replace=False)
sample_df = df[df['item_id'].isin(sample_items)]
monthly_trend = sample_df.groupby(['item_id','date'])[['value','weight','quantity']].sum().reset_index()
monthly_trend.head()